<a href="https://colab.research.google.com/github/zahraDehghanian97/AI_booklet_CE-AUT/blob/master/Densenet_Vs__ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mount google drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **prerequisit**

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import preprocessing
from tensorflow.keras.preprocessing import image
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Input, GlobalAveragePooling2D, Dropout ,  BatchNormalization
import glob
from sklearn.model_selection import train_test_split
from keras.models import Sequential
import os
%load_ext tensorboard

# **Make dataset ready**

In [3]:
def load_photos(type,dir_name):
    photo_list =[]
    y = []
    for file_name in (glob.glob(dir_name+'/*')):
        img = image.load_img(file_name, target_size=input_size)
        img = np.array(img)
        photo_list.append(img)
        y.append(type)
    return photo_list , y 
  
    
input_size = (128,128)
input_shape = (128,128,3)
dir_name_indoor = "/content/drive/MyDrive/Colab Notebooks/indoor"
dir_name_outdoor = "/content/drive/MyDrive/Colab Notebooks/outdoor"
X1 , y1= load_photos(0,dir_name_indoor) 
X2 , y2= load_photos(1,dir_name_outdoor) 
X1.extend(X2)
y1.extend(y2)
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.1, random_state=42)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
y_train = np.array(y_train)

[[132 135 114]
 [132 135 114]
 [130 133 112]
 [131 134 113]
 [131 134 113]
 [134 137 116]
 [132 135 114]
 [132 135 114]
 [134 137 116]
 [134 137 116]
 [134 137 116]
 [134 137 116]
 [134 137 116]
 [132 135 114]
 [132 135 114]
 [132 135 114]
 [131 134 113]
 [135 138 117]
 [135 138 117]
 [123 131 107]
 [135 143 119]
 [119 127 103]
 [135 135 123]
 [178 178 166]
 [223 223 211]
 [245 245 233]
 [235 246 242]
 [243 254 250]
 [227 238 234]
 [228 239 231]
 [213 224 216]
 [194 205 197]
 [168 182 157]
 [135 149 124]
 [118 132 107]
 [119 127 104]
 [119 127 104]
 [118 126 103]
 [118 126 103]
 [118 126 103]
 [116 124 101]
 [116 124 101]
 [116 124 101]
 [114 122  99]
 [114 122  99]
 [116 124 101]
 [114 122  99]
 [114 122  99]
 [109 121  97]
 [109 121  97]
 [109 121  97]
 [111 123  99]
 [111 123  99]
 [107 119  95]
 [107 119  95]
 [107 119  95]
 [107 119  95]
 [107 119  95]
 [109 121  97]
 [107 119  95]
 [107 119  95]
 [107 119  95]
 [107 119  95]
 [107 119  95]
 [107 119  97]
 [107 119  97]
 [105 117 

# **ResNet**

In [4]:
# keras.backend.clear_session()

# inputs = keras.Input(shape=input_shape)
# x = keras.applications.resnet.preprocess_input(inputs)
# base_model = keras.applications.ResNet152(include_top=False, weights="imagenet")
# base_model.trainable = False
# x = base_model(x) 
# # x = Flatten()(x)
# x = Dense(1000, activation='ReLU')(x)
# x = BatchNormalization()(x)
# x = Dense(250, activation='ReLU')(x)
# x = Dense(50, activation='ReLU')(x)
# x = Dense(10, activation='ReLU')(x)
# outputs = Dense(2, activation='softmax')(x)
# # outputs = GlobalAveragePooling2D()(x) 

# resnet_model = keras.Model(inputs, outputs)
# resnet_model.summary()



In [5]:
# # Inputs
# inputs = Input(shape=(128,128,3))
# # ResNet
# resnet152 = keras.applications.ResNet152(include_top=False, weights='imagenet')(inputs)
# # Our FC layer
# flat1 = Flatten()(resnet152)
# dense1 = Dense(units=128, use_bias=True)(flat1)
# batchnorm1 = BatchNormalization()(dense1)
# # Output
# out = Dense(units=1, activation='sigmoid')(batchnorm1)
# # Create Model
# model = keras.Model(inputs=inputs, outputs=out)
# model.summary()
# # model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00001), loss='binary_crossentropy',metrics=['accuracy'])

In [16]:
keras.backend.clear_session()
base_model = keras.applications.ResNet152(include_top=False, weights="imagenet",  input_shape=input_shape)
base_model.trainable  = False
model_res = Sequential()
model_res.add(base_model)
model_res.add(Flatten())
model_res.add(Dense(1024,activation='relu'))
# model_res.add(Dropout(0.2))
model_res.add(Dense(350,activation='relu'))
# model_res.add(Dropout(0.1))
model_res.add(Dense(128,activation='relu'))
# model_res.add(Dropout(0.1))
model_res.add(Dense(35,activation='relu'))
model_res.add(Dense(10,activation='relu'))
model_res.add(Dense(2))
# tf.keras.utils.plot_model(model_res,show_shapes=True,expand_nested=True)

 

In [17]:
# %rm -rf "/content/drive/Mydrive/Colab Notebooks/logs"
import shutil
os.chdir("/content/drive/MyDrive/Colab Notebooks")
# shutil.rmtree('./logs')
os.mkdir("./logs")

In [ ]:
tb_callback = keras.callbacks.TensorBoard(log_dir="./logs",histogram_freq=1)
model_res.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model_res.fit(X_train,y_train,epochs=50,validation_split=0.2,callbacks=[tb_callback])

Epoch 1/50
18/18 [==============================] - 124s 6s/step - loss: 7.7226 - accuracy: 0.6839 - val_loss: 0.3737 - val_accuracy: 0.9097
Epoch 2/50
18/18 [==============================] - 103s 6s/step - loss: 0.1040 - accuracy: 0.9711 - val_loss: 0.4963 - val_accuracy: 0.9167
Epoch 3/50
18/18 [==============================] - 104s 6s/step - loss: 0.0488 - accuracy: 0.9857 - val_loss: 0.5687 - val_accuracy: 0.9097
Epoch 4/50
18/18 [==============================] - 104s 6s/step - loss: 0.0088 - accuracy: 0.9984 - val_loss: 0.6053 - val_accuracy: 0.9028
Epoch 5/50
18/18 [==============================] - 103s 6s/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.6632 - val_accuracy: 0.9097
Epoch 6/50
18/18 [==============================] - 103s 6s/step - loss: 7.8460e-04 - accuracy: 1.0000 - val_loss: 0.6575 - val_accuracy: 0.9097
Epoch 7/50
18/18 [==============================] - 103s 6s/step - loss: 1.2182e-04 - accuracy: 1.0000 - val_loss: 0.6533 - val_accuracy: 0.9097
Epoch

In [ ]:
%tensorboard --logdir logs